In [1]:
##กรมบังคับคดี
from urllib.parse import quote
from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import csv

options = webdriver.ChromeOptions() 
options.add_argument("headless")
driver = webdriver.Chrome('/usr/bin/chromedriver',chrome_options=options)   
    
def find_percen_province(province):
    url = 'http://asset.led.go.th/newbidreg/'
    driver.get(url)
 
    driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div/div[3]/div/div[1]/table/tbody/tr/td[2]/select').send_keys(province)

    prov_percen = driver.current_url.split('=')[-1]

    return prov_percen

def find_maxpage(province):
    prov_percen = find_percen_province(province)
    
    page = 1
    url = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search=ok&search_asset_type_id=&search_tumbol=&search_ampur=&search_province=' + prov_percen + '&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_quaterrai=&search_wa=&search_rai_if=1&search_quaterrai_if=1&search_wa_if=1&search_status=1&search_person1=&page='+str(page)

    driver.get(url)
    max_page = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[2]/table[1]/tbody/tr/td[2]/div').text.split('/')[-1]
    print('max_page : ',max_page)
    return int(max_page)

# def scraping(prov_percen,max_page):
    
#             try:
#                 with open(csv_file, 'a') as csvfile:
#                     writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
# #                     writer.writeheader()
# #                     for d in prop_data:
#                     writer.writerow(prop_data)
#             except IOError:
#                 print("I/O error")    
                


def read_table_data(province,page):
    prov_percen = find_percen_province(province)
    dict_data = []
#     csv_file = "x.csv"
#     for page in range(1,max_page):
    print('page : ',page)
    url = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search=ok&search_asset_type_id=&search_tumbol=&search_ampur=&search_province=' + prov_percen + '&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_quaterrai=&search_wa=&search_rai_if=1&search_quaterrai_if=1&search_wa_if=1&search_status=1&search_person1=&page='+str(page)

    driver.get(url)

    html = driver.page_source
    # print(html)

    sup = BeautifulSoup(html, "html.parser").find("table",{"class":"table linkevent"}).find_all('tr')[2:]
    # print(sup)

    for i in sup:
        link = 'http://asset.led.go.th/newbidreg/'
        link += i['onclick'][13:-130].replace("ผบ", "%BC%BA").replace('บางส่วน','%BA%D2%A7%CA%E8%C7%B9').replace(' ','%20').replace('ไม่ปรากฏเลขทะเบียน','%E4%C1%E8%BB%C3%D2%A1%AF%E0%C5%A2%B7%D0%E0%BA%D5%C2%B9').replace('ม','C1')
        print(link)

        Law_Court_ID = link.split('&')[-3].split('=')[-1]
        deed_no = link.split('&')[-2].split('=')[-1].replace('%20','')
        addrno = link.split('&')[-1].split('=')[-1]


        ii = i.find_all('td')
        data = []
        for x in ii:
            data.append(x.text.strip())

        prop_data = {
            'Law_Court_ID' : Law_Court_ID,
            'deed_no' : deed_no,
            'addrno' : addrno,
            'year' : data[0],
            'order_sell' : data[1],
            'case_id' : data[2],
            'prop_type' : data[3],
            'size1' : data[4],
            'size2' : data[5],
            'size3' : data[6],
            'price' : data[7],
            'tumbon' : data[8],
            'aumper' : data[9],
            'link' : link
        }
#             'Law_Court_ID','deed_no','addrno','year','order_sell','case_id','prop_type','size1','size2','size3','price','tumbon','aumper','link'
        print(prop_data)
        print('------------------')
        dict_data.append(prop_data)

    return dict_data
    
def write_head_csv(file_name,csv_columns):
    ##write csv head
#     csv_columns = ['Law_Court_ID','deed_no','addrno','year','order_sell','case_id','prop_type','size1','size2','size3','price','tumbon','aumper','link']
    csv_file = file_name
    with open(csv_file, 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()

def write_csv(file_name,dict_data):
    try:
        with open(file_name, 'a') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
#             writer.writeheader()
            for data in dict_data:
                writer.writerow(data)
    except IOError:
        print("I/O error")
        
#main    
province = 'นนทบุรี'
file_name = "p.csv"
csv_columns = ['Law_Court_ID','deed_no','addrno','year','order_sell','case_id','prop_type','size1','size2','size3','price','tumbon','aumper','link']

write_head_csv(file_name,csv_columns)
max_page = find_maxpage(province)

for page_number in range(1,max_page):
    page_data = read_table_data(province,page_number)
    write_csv(file_name,page_data)

    

<ipython-input-1-db180efeb529>:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('/usr/bin/chromedriver',chrome_options=options)


max_page :  31
page :  1
http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.14800&law_suit_year=2561&Law_Court_ID=121&deed_no=160385,160386,160387,160388,160389,160390,160393,160394,160395,160396,160397,160&addrno=351/56
{'Law_Court_ID': '121', 'deed_no': '160385,160386,160387,160388,160389,160390,160393,160394,160395,160396,160397,160', 'addrno': '351/56', 'year': '4/2564 - 4/2564', 'order_sell': '7 - 1', 'case_id': 'ผบ.14800/2561', 'prop_type': 'ห้องชุด', 'size1': '0', 'size2': '0', 'size3': '33.44', 'price': '367,840.00', 'tumbon': 'บางบัวทอง', 'aumper': 'บางบัวทอง', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.14800&law_suit_year=2561&Law_Court_ID=121&deed_no=160385,160386,160387,160388,160389,160390,160393,160394,160395,160396,160397,160&addrno=351/56'}
------------------
http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.508&law_suit_year=2555&Law_Court_ID=101&deed_no=41299&addrno=189/219
{'Law_Court_ID': '101', 'deed

page :  2
http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66558&addrno=-
{'Law_Court_ID': '101', 'deed_no': '66558', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '64', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66558&addrno=-'}
------------------
http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66542&addrno=-
{'Law_Court_ID': '101', 'deed_no': '66542', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '19 - 4', 'case_id': 'ผบ.1680/2552', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '0', 'size2': '0', 'size3': '78', 'price': '58,420,000.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'li

page :  3
http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=ย.31&law_suit_year=2561&Law_Court_ID=101&deed_no=37932&addrno=99/277
{'Law_Court_ID': '101', 'deed_no': '37932', 'addrno': '99/277', 'year': '4/2564 - 4/2564', 'order_sell': '93 - 1', 'case_id': 'ย.31/2561', 'prop_type': 'ที่ดินพร้อมสิ่งปลูกสร้าง', 'size1': '0', 'size2': '0', 'size3': '76.1', 'price': '6,485,500.00', 'tumbon': 'บางพลับ', 'aumper': 'ปากเกร็ด', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=ย.31&law_suit_year=2561&Law_Court_ID=101&deed_no=37932&addrno=99/277'}
------------------
http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1103&law_suit_year=2559&Law_Court_ID=101&deed_no=151795&addrno=99/168
{'Law_Court_ID': '101', 'deed_no': '151795', 'addrno': '99/168', 'year': '4/2564 - 4/2564', 'order_sell': '95 - 1', 'case_id': 'ผบ.1103/2559', 'prop_type': 'ที่ดินพร้อมสิ่งปลูกสร้าง', 'size1': '0', 'size2': '0', 'size3': '21', 'price': '840,420.00', 'tumbon': 'ลำโพ', 'aum

page :  4
http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=1437&law_suit_year=2538&Law_Court_ID=101&deed_no=38294&addrno=-
{'Law_Court_ID': '101', 'deed_no': '38294', 'addrno': '-', 'year': '4/2564 - 4/2564', 'order_sell': '159 - 1', 'case_id': '1437/2538', 'prop_type': 'ที่ดินว่างเปล่า', 'size1': '11', 'size2': '0', 'size3': '11', 'price': '616,710,000.00', 'tumbon': 'บางพูด', 'aumper': 'ปากเกร็ด', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=1437&law_suit_year=2538&Law_Court_ID=101&deed_no=38294&addrno=-'}
------------------
http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=1437&law_suit_year=2538&Law_Court_ID=101&deed_no=32510&addrno=%E4%C1%E8%BB%C3%D2%A1%AF%E0%C5%A2%B7%D0%E0%BA%D5%C2%B9
{'Law_Court_ID': '101', 'deed_no': '32510', 'addrno': '%E4%C1%E8%BB%C3%D2%A1%AF%E0%C5%A2%B7%D0%E0%BA%D5%C2%B9', 'year': '4/2564 - 4/2564', 'order_sell': '159 - 1', 'case_id': '1437/2538', 'prop_type': 'ที่ดินพร้อมสิ่งปลูกสร้าง', 'size1': '4', 'size2': '3'

page :  5
http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.2094&law_suit_year=2562&Law_Court_ID=101&deed_no=172284&addrno=239/227
{'Law_Court_ID': '101', 'deed_no': '172284', 'addrno': '239/227', 'year': '7/2564 - 7/2564', 'order_sell': '69 - 1', 'case_id': 'ผบ.2094/2562', 'prop_type': 'ที่ดินพร้อมสิ่งปลูกสร้าง', 'size1': '0', 'size2': '0', 'size3': '52.5', 'price': '1,658,958.75', 'tumbon': 'พิมลราช', 'aumper': 'บางบัวทอง', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.2094&law_suit_year=2562&Law_Court_ID=101&deed_no=172284&addrno=239/227'}
------------------
http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.825&law_suit_year=2563&Law_Court_ID=101&deed_no=189519&addrno=59/204
{'Law_Court_ID': '101', 'deed_no': '189519', 'addrno': '59/204', 'year': '7/2564 - 7/2564', 'order_sell': '70 - 1', 'case_id': 'ผบ.825/2563', 'prop_type': 'ที่ดินพร้อมสิ่งปลูกสร้าง', 'size1': '0', 'size2': '0', 'size3': '25.3', 'price': '1,186,768.0

page :  6
http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.8451&law_suit_year=2557&Law_Court_ID=145&deed_no=74203&addrno=71/276
{'Law_Court_ID': '145', 'deed_no': '74203', 'addrno': '71/276', 'year': '7/2564 - 7/2564', 'order_sell': '142 - 1', 'case_id': 'ผบ.8451/2557', 'prop_type': 'ที่ดินพร้อมสิ่งปลูกสร้าง', 'size1': '0', 'size2': '0', 'size3': '16', 'price': '953,750.00', 'tumbon': 'บางแม่นาง', 'aumper': 'บางใหญ่', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.8451&law_suit_year=2557&Law_Court_ID=145&deed_no=74203&addrno=71/276'}
------------------
http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1622&law_suit_year=2563&Law_Court_ID=101&deed_no=108399&addrno=79/16%20และ%2079/17
{'Law_Court_ID': '101', 'deed_no': '108399', 'addrno': '79/16%20และ%2079/17', 'year': '7/2564 - 7/2564', 'order_sell': '143 - 1', 'case_id': 'ผบ.1622/2563', 'prop_type': 'ที่ดินพร้อมสิ่งปลูกสร้าง', 'size1': '0', 'size2': '0', 'size3': '62.3', '

page :  7
http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.5108&law_suit_year=2562&Law_Court_ID=010&deed_no=124949&addrno=91/242
{'Law_Court_ID': '010', 'deed_no': '124949', 'addrno': '91/242', 'year': '7/2564 - 7/2564', 'order_sell': '191 - 1', 'case_id': 'ผบ.5108/2562', 'prop_type': 'ที่ดินพร้อมสิ่งปลูกสร้าง', 'size1': '0', 'size2': '0', 'size3': '16', 'price': '892,800.00', 'tumbon': 'บางบัวทอง', 'aumper': 'บางบัวทอง', 'link': 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.5108&law_suit_year=2562&Law_Court_ID=010&deed_no=124949&addrno=91/242'}
------------------
http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.50&law_suit_year=2552&Law_Court_ID=706&deed_no=141989&addrno=90/3
{'Law_Court_ID': '706', 'deed_no': '141989', 'addrno': '90/3', 'year': '7/2564 - 7/2564', 'order_sell': '192 - 1', 'case_id': 'ผบ.50/2552', 'prop_type': 'ที่ดินพร้อมสิ่งปลูกสร้าง', 'size1': '0', 'size2': '0', 'size3': '42.1', 'price': '1,449,412.50', 'tumb

KeyboardInterrupt: 

In [3]:
#ค้นหาแปลงที่ดิน
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions() 
options.add_argument("headless")
driver = webdriver.Chrome('/usr/bin/chromedriver',chrome_options=options)


def find_gps(province,aumpur,deed_no):
#     province = 'นนทบุรี'
#     aumpur = '02 - บางกรวย'
#     deed_no = '9805'

    # driver_path = "/Users/amit/Downloads/chromedriver"
    url = 'http://dolwms.dol.go.th/tvwebp/'

    driver.get(url)


    driver.find_element_by_xpath('//*[@id="form1"]/div[6]/div/a[2]').click()
    # time.sleep(1)
    driver.find_element_by_xpath('//*[@id="ddlProvince"]').send_keys(province)
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="ddlAmphur"]').send_keys(aumpur)
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="txtPacelNo"]').send_keys(deed_no)
    # time.sleep(2)
    driver.find_element_by_xpath('//*[@id="btnFind"]').click()

    time.sleep(1)
    # driver.page_source
    # driver.find_element(By.XPATH, '//*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div/a[1]').text
    d = driver.find_element(By.XPATH, '//*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div').text

    d = d.split('\n')
#     print(d)

    map_data = {
        'lat' : d[-6].split(',')[0].split(':')[1].strip(),
        'long' : d[-6].split(',')[1],
        'price_gov' : d[-8].split()[2],
        'area1' : d[-12].split()[2],
        'area2' : d[-12].split()[4],
        'area3' : d[-12].split()[6],
    }

    return map_data


province = 'นนทบุรี'
aumpur = '02 - บางกรวย'
deed_no = '9805'
    
# print(find_gps(province,aumpur,deed_no))
find_gps('นนทบุรี', '06 - ปากเกร็ด', '68557')
    

# driver.get_attribute("innerHTML")


# //*[@id="ddlProvince"] นนทบุรี    12
# //*[@id="ddlAmphur"] 02 - บางกรวย      02
# //*[@id="txtPacelNo"]   10554
# //*[@id="btnFind"]   
# //*[@id="btnFind"]
# //*[@id="btnFind"]

# //*[@id="form1"]/div[6]/div/a[2]

# //*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div

# //*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div/a[3]

<ipython-input-3-091fcb247930>:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('/usr/bin/chromedriver',chrome_options=options)


{'lat': '13.9361549675151',
 'long': '100.5123114300',
 'price_gov': 'กรุณาตรวจสอบข้อมูลผ่านทาง',
 'area1': '0',
 'area2': '0',
 'area3': '18.7'}

In [ ]:
##กรมบังคับคดี
from urllib.parse import quote
from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import csv

options = webdriver.ChromeOptions() 
options.add_argument("headless")
driver = webdriver.Chrome('/usr/bin/chromedriver',chrome_options=options)

csv_columns = ['Law_Court_ID','deed_no','addrno','year','order_sell','case_id','prop_type','size1','size2','size3','price','tumbon','aumper','link']
csv_file = "x.csv"
with open(csv_file, 'a') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()
    
    
def find_percen_province(province):
    url = 'http://asset.led.go.th/newbidreg/'
    driver.get(url)
 
    driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div/div[3]/div/div[1]/table/tbody/tr/td[2]/select').send_keys(province)

    prov_percen = driver.current_url.split('=')[-1]

    return prov_percen

def find_maxpage(prov_percen):
    
    page = 1
    url = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search=ok&search_asset_type_id=&search_tumbol=&search_ampur=&search_province=' + prov_percen + '&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_quaterrai=&search_wa=&search_rai_if=1&search_quaterrai_if=1&search_wa_if=1&search_status=1&search_person1=&page='+str(page)

    driver.get(url)
    max_page = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[2]/table[1]/tbody/tr/td[2]/div').text.split('/')[-1]
    print('max_page : ',max_page)
    return int(max_page)

def scraping(prov_percen,max_page):
#     dict_data = []
#     csv_file = "x.csv"
    for page in range(1,max_page):
        print('page : ',page)
        url = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search=ok&search_asset_type_id=&search_tumbol=&search_ampur=&search_province=' + prov_percen + '&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_quaterrai=&search_wa=&search_rai_if=1&search_quaterrai_if=1&search_wa_if=1&search_status=1&search_person1=&page='+str(page)

        driver.get(url)

        html = driver.page_source
        # print(html)

        sup = BeautifulSoup(html, "html.parser").find("table",{"class":"table linkevent"}).find_all('tr')[2:]
        # print(sup)

        for i in sup:
            link = 'http://asset.led.go.th/newbidreg/'
            link += i['onclick'][13:-130].replace("ผบ", "%BC%BA").replace('บางส่วน','%BA%D2%A7%CA%E8%C7%B9').replace(' ','%20').replace('ไม่ปรากฏเลขทะเบียน','%E4%C1%E8%BB%C3%D2%A1%AF%E0%C5%A2%B7%D0%E0%BA%D5%C2%B9').replace('ม','C1')
            print(link)

            Law_Court_ID = link.split('&')[-3].split('=')[-1]
            deed_no = link.split('&')[-2].split('=')[-1].replace('%20','')
            addrno = link.split('&')[-1].split('=')[-1]


            ii = i.find_all('td')
            data = []
            for x in ii:
                data.append(x.text.strip())

            prop_data = {
                'Law_Court_ID' : Law_Court_ID,
                'deed_no' : deed_no,
                'addrno' : addrno,
                'year' : data[0],
                'order_sell' : data[1],
                'case_id' : data[2],
                'prop_type' : data[3],
                'size1' : data[4],
                'size2' : data[5],
                'size3' : data[6],
                'price' : data[7],
                'tumbon' : data[8],
                'aumper' : data[9],
                'link' : link
            }
#             'Law_Court_ID','deed_no','addrno','year','order_sell','case_id','prop_type','size1','size2','size3','price','tumbon','aumper','link'
            print(prop_data)
            print('------------------')
            try:
                with open(csv_file, 'a') as csvfile:
                    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
#                     writer.writeheader()
#                     for d in prop_data:
                    writer.writerow(prop_data)
            except IOError:
                print("I/O error")    
    
prov_percen = find_percen_province('นนทบุรี')
max_page = find_maxpage(prov_percen)
scraping(prov_percen,max_page)
    
    
    
    
    
    
    

    
    



            
#             dict_data.append(prop_data) 
    
#             try:
#                 with open(csv_file, 'a') as csvfile:
#                     writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
#                     writer.writeheader()
#                     for d in prop_data:
#                         writer.writerow(d)
#             except IOError:
#                 print("I/O error")   
# d = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[2]/div[2]/table/tbody').text.split('\n')


# links = driver.find_elements_by_partial_link_text('##')
# for link in links:
#     print('lol : ',link.get_attribute("href"))


#     print('Law_Court_ID : ',Law_Court_ID)
#     print('deed_no : ',deed_no)
#     print('addrno : ',addrno)    
    
  # driver.find_element_by_xpath('//*[@id="form1"]/div[6]/div/a[2]').click()
    # time.sleep(1)
    # driver.find_element_by_xpath('//*[@id="ddlProvince"]').send_keys('นนทบุรี')
    # driver.find_element_by_xpath('//*[@id="prov"]').click()
    # time.sleep(1)

# driver.find_element_by_css_selector('#tbody')



# for i in d:
#     print(i.split())
#     data = i.split()
#     prop_data = {
#         'Law_Court_ID' : data[6],
#         'deed_no' : data[0],
#         'addrno' : data[0],
#         'year' : data[0],
#         'order_sell' : data[1],
#         'case_id' : data[2],
#         'prop_type' : data[3],
#         'size1' : data[4],
#         'size2' : data[5],
#         'size3' : data[6],
#         'price' : data[7],
#         'tumbon' : data[8],
#         'aumper' : data[9],
# #         'link' : link
#     }
#     print(prop_data)






# page = 1
# try :
# # url_naweb = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search_asset_type_id=&search_tumbol=&search_ampur=&search_province=' + prov_percen + '&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_rai_if=1&search_quaterrai=&search_quaterrai_if=1&search_wa=&search_wa_if=1&search_status=1&search_person1=&search=ok'
# url_naweb = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search=ok&search_asset_type_id=&search_tumbol=&search_ampur=&search_province=' + prov_percen + '&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_quaterrai=&search_wa=&search_rai_if=1&search_quaterrai_if=1&search_wa_if=1&search_status=1&search_person1=&page='+str(page)
# print(url_naweb)

# r = requests.get(url_naweb)
# r.encoding = 'utf-8' # กำหนด encoding กันปัญหาการถอดรหัสผิดแบบ


# sup = BeautifulSoup(r.content, 'html.parser').find("table",{"class":"table linkevent"}).find_all('tr')[2:]

# for i in sup:
#     link = 'http://asset.led.go.th/newbidreg/'
#     link += i['onclick'][13:-130].replace("ผบ", "%BC%BA").replace(' ','%20')
#     print(link)
#     Law_Court_ID = link.split('&')[-3].split('=')[-1]
#     deed_no = link.split('&')[-2].split('=')[-1]
#     addrno = link.split('&')[-1].split('=')[-1]
    
#     ii = i.find_all('td')
#     data = []
#     for x in ii:
#         data.append(x.text.strip())
#     prop_data = {
#         'Law_Court_ID' : Law_Court_ID,
#         'deed_no' : deed_no,
#         'addrno' : addrno,
#         'year' : data[0],
#         'order_sell' : data[1],
#         'case_id' : data[2],
#         'prop_type' : data[3],
#         'size1' : data[4],
#         'size2' : data[5],
#         'size3' : data[6],
#         'price' : data[7],
#         'tumbon' : data[8],
#         'aumper' : data[9],
#         'link' : link
#     }
#     print(prop_data)
#     print('------------------')












<ipython-input-3-648e233cb593>:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('/usr/bin/chromedriver',chrome_options=options)


In [ ]:
# http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66422&addrno=%E4%C1%E8%BB%C3%D2%A1%AF%E0%C5%A2%B7%D0%E0%BA%D5%C2%B9
# http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66422&addrno=ไม่ปรากฏเลขทะเบียน  
    
# http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.4280&law_suit_year=2561&Law_Court_ID=121&deed_no=149458&addrno=78/475%20ม.9
# http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.4280&law_suit_year=2561&Law_Court_ID=121&deed_no=149458&addrno=78/475%20%C1.9   
    
# http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.18670&law_suit_year=2560&Law_Court_ID=121&deed_no=178896,178897%20(%BA%D2%A7%CA%E8%C7%B9)&addrno=9/188
# http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.18670&law_suit_year=2560&Law_Court_ID=121&deed_no=178896,178897%20(บางส่วน)&addrno=9/188    

In [ ]:
##กรมบังคับคดี
from urllib.parse import quote
from bs4 import BeautifulSoup
import requests
url_naweb = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search_asset_type_id=&search_tumbol=&search_ampur=&search_province=%B9%B9%B7%BA%D8%C3%D5&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_rai_if=1&search_quaterrai=&search_quaterrai_if=1&search_wa=&search_wa_if=1&search_status=1&search_person1=&search=ok'

r = requests.get(url_naweb)
r.encoding = 'utf-8' # กำหนด encoding กันปัญหาการถอดรหัสผิดแบบ

sup = BeautifulSoup(r.content, 'html.parser').find("table",{"class":"table linkevent"}).find_all('tr')[2:]

for i in sup:
    link = 'http://asset.led.go.th/newbidreg/'
    link += i['onclick'][13:-130].replace("ผบ", "%BC%BA").replace(' ','%20')
#     print(link)
    Law_Court_ID = link.split('&')[-3].split('=')[-1]
    deed_no = link.split('&')[-2].split('=')[-1]
    addrno = link.split('&')[-1].split('=')[-1]
#     print('Law_Court_ID : ',Law_Court_ID)
#     print('deed_no : ',deed_no)
#     print('addrno : ',addrno)
    
    ii = i.find_all('td')
    data = []
    for x in ii:
        data.append(x.text.strip())
#     print(data)
    prop_data = {
        'Law_Court_ID' : Law_Court_ID,
        'deed_no' : deed_no,
        'addrno' : addrno,
        'year' : data[0],
        'order_sell' : data[1],
        'case_id' : data[2],
        'prop_type' : data[3],
        'size1' : data[4],
        'size2' : data[5],
        'size3' : data[6],
        'price' : data[7],
        'tumbon' : data[8],
        'aumper' : data[9],
        'link' : link
    }
    print(prop_data)
    print('------------------')

In [2]:
#ค้นหาแปลงที่ดิน
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions() 
options.add_argument("headless")
driver = webdriver.Chrome('/usr/bin/chromedriver',chrome_options=options)


def find_gps(province,aumpur,deed_no):
#     province = 'นนทบุรี'
#     aumpur = '02 - บางกรวย'
#     deed_no = '9805'

    # driver_path = "/Users/amit/Downloads/chromedriver"
    url = 'http://dolwms.dol.go.th/tvwebp/'

    driver.get(url)


    driver.find_element_by_xpath('//*[@id="form1"]/div[6]/div/a[2]').click()
    # time.sleep(1)
    driver.find_element_by_xpath('//*[@id="ddlProvince"]').send_keys(province)
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="ddlAmphur"]').send_keys(aumpur)
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="txtPacelNo"]').send_keys(deed_no)
    # time.sleep(2)
    driver.find_element_by_xpath('//*[@id="btnFind"]').click()

    time.sleep(1)
    # driver.page_source
    # driver.find_element(By.XPATH, '//*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div/a[1]').text
    d = driver.find_element(By.XPATH, '//*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div').text

    d = d.split('\n')
#     print(d)

    map_data = {
        'lat' : d[-6].split(',')[0].split(':')[1].strip(),
        'long' : d[-6].split(',')[1],
        'price_gov' : d[-8].split()[2],
        'area1' : d[-12].split()[2],
        'area2' : d[-12].split()[4],
        'area3' : d[-12].split()[6],
    }

    return map_data


province = 'นนทบุรี'
aumpur = '02 - บางกรวย'
deed_no = '9805'
    
print(find_gps(province,aumpur,deed_no))
    

# driver.get_attribute("innerHTML")


# //*[@id="ddlProvince"] นนทบุรี    12
# //*[@id="ddlAmphur"] 02 - บางกรวย      02
# //*[@id="txtPacelNo"]   10554
# //*[@id="btnFind"]   
# //*[@id="btnFind"]
# //*[@id="btnFind"]

# //*[@id="form1"]/div[6]/div/a[2]

# //*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div

# //*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div/a[3]

<ipython-input-2-fde47fb7f3de>:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('/usr/bin/chromedriver',chrome_options=options)


{'lat': '13.8287281871677', 'long': '100.3945950334', 'price_gov': '18,000', 'area1': '0', 'area2': '0', 'area3': '76.5'}


In [ ]:
##link ok
from urllib.parse import quote
from bs4 import BeautifulSoup
import requests
url_naweb = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search_asset_type_id=&search_tumbol=&search_ampur=&search_province=%B9%B9%B7%BA%D8%C3%D5&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_rai_if=1&search_quaterrai=&search_quaterrai_if=1&search_wa=&search_wa_if=1&search_status=1&search_person1=&search=ok'

r = requests.get(url_naweb)
r.encoding = 'utf-8' # กำหนด encoding กันปัญหาการถอดรหัสผิดแบบ
sup = BeautifulSoup(r.text,'lxml').find("table",{"class":"table linkevent"}).find_all('tr')[2:]

for i in sup:
#     url_link = 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.' 
#     url_link += i['onclick'][12:-16].split(',')[0].split('.')[2].split("'")[0]
#     print(url_link)
    a = i['onclick'][43:-130].replace(" ", "%20")
#     print(a)
    print('http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.'+a)

In [ ]:
##link ok
from urllib.parse import quote
from bs4 import BeautifulSoup
import requests
url_naweb = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search_asset_type_id=&search_tumbol=&search_ampur=&search_province=%B9%B9%B7%BA%D8%C3%D5&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_rai_if=1&search_quaterrai=&search_quaterrai_if=1&search_wa=&search_wa_if=1&search_status=1&search_person1=&search=ok'

r = requests.get(url_naweb)
r.encoding = 'utf-8' # กำหนด encoding กันปัญหาการถอดรหัสผิดแบบ

sup = BeautifulSoup(r.content, 'html.parser').find("table",{"class":"table linkevent"}).find_all('tr')[2:]

for i in sup:
    link = 'http://asset.led.go.th/newbidreg/'
    link += i['onclick'][13:-130].replace("ผบ", "%BC%BA").replace(' ','%20')
#     print(link)
    Law_Court_ID = link.split('&')[-3].split('=')[-1]
    deed_no = link.split('&')[-2].split('=')[-1]
    addrno = link.split('&')[-1].split('=')[-1]
#     print('Law_Court_ID : ',Law_Court_ID)
#     print('deed_no : ',deed_no)
#     print('addrno : ',addrno)
    
    ii = i.find_all('td')
    data = []
    for x in ii:
        data.append(x.text.strip())
#     print(data)
    prop_data = {
        'Law_Court_ID' : Law_Court_ID,
        'deed_no' : deed_no,
        'addrno' : addrno,
        'year' : data[0],
        'order_sell' : data[1],
        'case_id' : data[2],
        'prop_type' : data[3],
        'size1' : data[4],
        'size2' : data[5],
        'size3' : data[6],
        'price' : data[7],
        'tumbon' : data[8],
        'aumper' : data[9],
        'link' : link
    }
    print(prop_data)
    print(link)
    print('------------------')


In [ ]:
http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.508&law_suit_year=2555&Law_Court_ID=101&deed_no=41299&addrno=189/219


len("','_blank',null,'width=1300,height=900,status=yes,toolbar=no,menubar=yes,location=no,scrollbars=yes,resizable=yes'); return false;")

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
# url = requests.get("https://pantip.com/topic/37319282")
url = requests.get("http://asset.led.go.th/newbidreg/asset_search_province.asp?search_asset_type_id=&search_tumbol=&search_ampur=&search_province=%B9%B9%B7%BA%D8%C3%D5&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_rai_if=1&search_quaterrai=&search_quaterrai_if=1&search_wa=&search_wa_if=1&search_status=1&search_person1=&search=ok")
soup = BeautifulSoup(url.content, "html.parser")

In [ ]:
data

In [ ]:
import requests
from bs4 import BeautifulSoup

# r = requests.get("https://coins.co.th/")
# soup = BeautifulSoup(r.content, "html.parser")

url = requests.get("http://asset.led.go.th/newbidreg/asset_search_province.asp?search_asset_type_id=&search_tumbol=&search_ampur=&search_province=%B9%B9%B7%BA%D8%C3%D5&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_rai_if=1&search_quaterrai=&search_quaterrai_if=1&search_wa=&search_wa_if=1&search_status=1&search_person1=&search=ok")
soup = BeautifulSoup(url.content, "html.parser")

data = soup.find_all("tbody")
print(data)

In [ ]:
from urllib.parse import quote
from bs4 import BeautifulSoup
import requests
url_naweb = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search_asset_type_id=&search_tumbol=&search_ampur=&search_province=%B9%B9%B7%BA%D8%C3%D5&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_rai_if=1&search_quaterrai=&search_quaterrai_if=1&search_wa=&search_wa_if=1&search_status=1&search_person1=&search=ok'

r = requests.get(url_naweb)
r.encoding = 'utf-8' # กำหนด encoding กันปัญหาการถอดรหัสผิดแบบ
sup = BeautifulSoup(r.text,'lxml').find("table",{"class":"table linkevent"}).find_all('tr')[2:]

for i in sup:
#     url_link = 'http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.' 
#     url_link += i['onclick'][12:-16].split(',')[0].split('.')[2].split("'")[0]
#     print(url_link)
    a = i['onclick'][43:-130].replace(" ", "%20")
#     print(a)
    print('http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.'+a)
#     a = quote('deed_no=41510  41511  41514',safe='=')

    print('--------------------------------')

# print(type(sup))

# print(sup.find_all('table'))
# print(sup)


# table class="table linkevent"

# data = soup.find("h2",{"class":"display-post-title"})
# deed_no=41510%20%2041511%20%2041514&addrno=132/42

In [ ]:
# soup = BeautifulSoup(html_doc, 'html.parser')
http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.14800&law_suit_year=2561&Law_Court_ID=121&deed_no=160385,160386,160387,160388,160389,160390,160393,160394,160395,160396,160397,160&addrno=351/56

In [ ]:
from urllib.parse import quote
a = quote('deed_no=41510  41511  41514',safe='=')
a
# deed_no=41510%20%2041511%20%2041514&addrno=132/42

In [ ]:
len(",'_blank',null,'width=1300,height=900,status=yes,toolbar=no,menubar=yes,location=no,scrollbars=yes,resizable=yes'); return false;")

In [ ]:
http://asset.led.go.th/newbidreg/asset_open.asp?law_suit_no=%BC%BA.1680&law_suit_year=2552&Law_Court_ID=101&deed_no=66522&addrno=-

In [ ]:
# https://stackoverflow.com/questions/53973423/getting-value-after-button-click-with-beautifulsoup-python

from selenium.webdriver.support.ui import WebDriverWait as wait

def open_browser():
    driver = webdriver.Chrome("/home/felipe/Downloads/chromedriver")
    driver.get(url)
    driver.find_element_by_id('bt_gerar_cpf').click()
    text_field = driver.find_element_by_id('texto_cpf')
    text = wait(driver, 10).until(lambda driver: not text_field.text == 'Gerando...' and text_field.text)
    return text

print(open_browser())

In [ ]:
import requests

url = 'https://www.4devs.com.br/ferramentas_online.php'
data = {'acao': 'gerar_cpf', 'pontuacao': 'S'}
response = requests.post(url, data=data)
print(response.text)

In [ ]:
import requests

url = 'http://dolwms.dol.go.th/tvwebp/'
data = {'ddlProvince':'12','ddlAmphur':'06','txtPacelNo': '41510'}
# นนทบุรี

response = requests.post(url, data=data)
print(response.text)

In [ ]:
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions() 
options.add_argument("headless")

driver = webdriver.Chrome('/usr/bin/chromedriver',chrome_options=options)

# driver_path = "/Users/amit/Downloads/chromedriver"
url = 'http://dolwms.dol.go.th/tvwebp/'

driver.get(url)


driver.find_element_by_xpath('//*[@id="form1"]/div[6]/div/a[2]').click()
# time.sleep(1)
driver.find_element_by_xpath('//*[@id="ddlProvince"]').send_keys('นนทบุรี')
time.sleep(1)
driver.find_element_by_xpath('//*[@id="ddlAmphur"]').send_keys('02 - บางกรวย')
time.sleep(1)
driver.find_element_by_xpath('//*[@id="txtPacelNo"]').send_keys('9805')
# time.sleep(2)
driver.find_element_by_xpath('//*[@id="btnFind"]').click()

time.sleep(1)
# driver.page_source
# driver.find_element(By.XPATH, '//*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div/a[1]').text
d = driver.find_element(By.XPATH, '//*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div').text

d = d.split('\n')
print(d)

map_data = {
    'lat' : d[-6].split(',')[0].split(':')[1].strip(),
    'long' : d[-6].split(',')[1],
    'price_gov' : d[-8].split()[2],
    'area1' : d[-12].split()[2],
    'area2' : d[-12].split()[4],
    'area3' : d[-12].split()[6],
}

map_data

# driver.get_attribute("innerHTML")


# //*[@id="ddlProvince"] นนทบุรี    12
# //*[@id="ddlAmphur"] 02 - บางกรวย      02
# //*[@id="txtPacelNo"]   10554
# //*[@id="btnFind"]   
# //*[@id="btnFind"]
# //*[@id="btnFind"]

# //*[@id="form1"]/div[6]/div/a[2]

# //*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div

# //*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div/a[3]

In [ ]:
# v1
# # import time

# # from selenium import webdriver
# # from selenium.webdriver.common.keys import Keys
# # from selenium.webdriver.common.by import By

# # driver = webdriver.Chrome('/usr/bin/chromedriver')

# # # driver_path = "/Users/amit/Downloads/chromedriver"
# # url = 'http://dolwms.dol.go.th/tvwebp/'

# # driver.get(url)


# # driver.find_element_by_xpath('//*[@id="form1"]/div[6]/div/a[2]').click()
# # # time.sleep(1)
# # driver.find_element_by_xpath('//*[@id="ddlProvince"]').send_keys('นนทบุรี')


# # time.sleep(1)
# # driver.find_element_by_xpath('//*[@id="ddlAmphur"]').send_keys('02 - บางกรวย')
# # time.sleep(1)
# # driver.find_element_by_xpath('//*[@id="txtPacelNo"]').send_keys('9805')
# # # time.sleep(2)
# # driver.find_element_by_xpath('//*[@id="btnFind"]').click()

# # time.sleep(1)
# # # driver.page_source
# # # driver.find_element(By.XPATH, '//*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div/a[1]').text
# # driver.find_element(By.XPATH, '//*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div').text

# # # driver.get_attribute("innerHTML")


# # # //*[@id="ddlProvince"] นนทบุรี    12
# # # //*[@id="ddlAmphur"] 02 - บางกรวย      02
# # # //*[@id="txtPacelNo"]   10554
# # # //*[@id="btnFind"]   
# # # //*[@id="btnFind"]
# # # //*[@id="btnFind"]

# # # //*[@id="form1"]/div[6]/div/a[2]

# # # //*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div

# # # //*[@id="map_canvas"]/div/div/div[1]/div[3]/div/div[4]/div/div/div/div/div/div/a[3]

In [7]:
a = 'ปากเกร็ด'
a.split('(')[0]

'ปากเกร็ด'